In [6]:
import simpy
import random
import numpy as np

# Simulation parameters
RANDOM_SEED = 42
NUM_STAFF = 50          # Number of staff available
SIM_TIME = 1440        # Simulation time in minutes (24 hours)
ARRIVAL_RATE = 6    # Patients per hour (Poisson distribution)
SERVICE_TIME_MEAN = 20 # Mean service time in minutes
MIN_WAIT_TIME = 15  # Minimum wait time in minutes

# Patient arrival process
def patient(env, name, staff, wait_times):
    arrival_time = env.now
    print(f"{name} arrives at {arrival_time:.2f}.")

    # Request a staff member
    with staff.request() as request:
        yield request
        # Ensure minimum wait time
        min_wait_end = max(arrival_time + MIN_WAIT_TIME, env.now)
        yield env.timeout(max(0, min_wait_end - env.now))
        
        # Record actual wait time
        wait_time = env.now - arrival_time
        wait_times.append(wait_time)
        
        print(f"{name} starts service at {env.now:.2f} after waiting {wait_time:.2f}.")
        
        # Service time
        service_time = random.expovariate(1 / SERVICE_TIME_MEAN)
        yield env.timeout(service_time)
        print(f"{name} leaves at {env.now:.2f}.")

# Generate patients
def patient_generator(env, staff, wait_times):
    i = 0
    while True:
        interarrival_time = random.expovariate(ARRIVAL_RATE / 60.0)
        yield env.timeout(interarrival_time)
        i += 1
        env.process(patient(env, f"Patient {i}", staff, wait_times))

# Main simulation
random.seed(RANDOM_SEED)
env = simpy.Environment()
staff = simpy.Resource(env, capacity=NUM_STAFF)
wait_times = []

env.process(patient_generator(env, staff, wait_times))
env.run(until=SIM_TIME)

# Results
print("\nSimulation Results:")
print(f"Average wait time: {np.mean(wait_times):.2f} minutes")
print(f"Minimum wait time: {np.min(wait_times):.2f} minutes")
print(f"Maximum wait time: {np.max(wait_times):.2f} minutes")
print(f"Total patients served: {len(wait_times)}")


Patient 1 arrives at 10.20.
Patient 2 arrives at 10.45.
Patient 3 arrives at 13.67.
Patient 4 arrives at 16.20.
Patient 1 starts service at 25.20 after waiting 15.00.
Patient 2 starts service at 25.45 after waiting 15.00.
Patient 3 starts service at 28.67 after waiting 15.00.
Patient 5 arrives at 29.53.
Patient 3 leaves at 30.49.
Patient 4 starts service at 31.20 after waiting 15.00.
Patient 4 leaves at 31.80.
Patient 6 arrives at 35.01.
Patient 7 arrives at 37.48.
Patient 8 arrives at 44.52.
Patient 5 starts service at 44.53 after waiting 15.00.
Patient 9 arrives at 44.79.
Patient 1 leaves at 47.78.
Patient 5 leaves at 48.97.
Patient 6 starts service at 50.01 after waiting 15.00.
Patient 7 starts service at 52.48 after waiting 15.00.
Patient 10 arrives at 55.28.
Patient 7 leaves at 57.46.
Patient 8 starts service at 59.52 after waiting 15.00.
Patient 9 starts service at 59.79 after waiting 15.00.
Patient 9 leaves at 59.92.
Patient 11 arrives at 64.18.
Patient 6 leaves at 65.76.
Patien